# Recurrent Neural Network(RNN)

## classification

In [1]:
import torch
import torch.nn as nn
import torchvision
import torch.utils.data as Data
import matplotlib.pyplot as plt

EPOCH = 1
BATCH_SIZE = 64
TIME_STEP = 28#表示每个输入样本有28个时间步
#如果输入是一个手写数字图片,可以把每一行作为一个时间步，总共28行
INPUT_SIZE = 28#每个时间步的输入向量的长度
LR = 0.01
DOWNLOAD_MNIST = False

In [ ]:
train_data = torchvision.datasets.MNIST(
    root='./mnist/',
    train=True,
    transform=torchvision.transforms.ToTensor(),
    download=DOWNLOAD_MNIST
)
train_loader = Data.DataLoader(
    dataset=train_data,
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_data = torchvision.datasets.MNIST(
    root='./mnist/',
    train=False,
    transform=torchvision.transforms.ToTensor(),
    download=DOWNLOAD_MNIST
)
test_x = test_data.data.type(torch.FloatTensor)[:2000]/255.0
test_y = test_data.targets[:2000]

class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        self.rnn = nn.LSTM(
            input_size=INPUT_SIZE,#每个时间点输入28个像素点
            hidden_size=64,#每个时间步会得到64个特征值
            num_layers=1,#表示堆叠了多少个 RNN/LSTM/GRU 层，即网络的深度
            batch_first=True#输入数据的格式为 (batch, time_step, input_size)（三个维度）
            #输入的时候是 (time_step, batch, input_size)（默认格式），所以手动改一下（不该也没关系只是习惯问题）
        )
        #LSTM 层输出(N, 64) （只用最后一个时间步的输出）然后作为输入输入到全连接层
        self.out = nn.Linear(64, 10)#10是分类任务的类别数量，MNIST 手写数字识别：数字 0~9
        #nn.Linear(in_features, out_features)

    def forward(self, x):#x 是你送入神经网络的数据
        #x 的形状是 (batch, time_step, input_size)
        r_out, (h_n,h_c) = self.rnn(x, None)#（none表示初始隐藏状态是有是无）
        out = self.out(r_out[:, -1, :])#选取最后一个时刻的r_out进行最终的输出
        #r_out 的形状是 (batch, time step, hidden_size)，我们要选取最后一个时刻的r_out，所以就中间值置为-1
        return out

num_layers=1：只有一层 RNN/LSTM/GRU，
输入数据只被RNN单层处理一次，输出就是这一层的结果。  
num_layers=2：有两层，第一层的输出会作为第二层的输入，依次类推，形成多层结构。

num_layers=1：单层RNN/LSTM/GRU，结构简单，参数较少。  
增加 num_layers 可以提升模型表达能力，但也增加计算量和过拟合风险。

隐藏层神经元个数：  
LSTM 的输出会作为后续层（如全连接层）的输入，其维度就是 hidden_size。比如你最后分类用 nn.Linear(hidden_size, num_classes)，这里输入维度就是64。  
hidden_size 越大，模型可以表示越复杂的序列关系，能学习到更多信息。但太大容易过拟合，太小可能欠拟合。

r_out, (h_n,h_c) = self.rnn(x, None)：  
输入的shape是(batch,time_step,input_size)  
第k个step执行完之后会输出(h_n,h_c)（即hidden state）然后它和第k+1个step中的input_size共同产生第k+1个step的rnn_out，同时会产生k+1个step的hidden state，然后他再和第k+2个step的input_size共同产生……

hidden state中有两个参数：h_c和h_n  
我们用不到最后一个step的hidden state，我们要的是最后一个step的r_out

我们取最后一个时刻的r_out作为输入，也就是我们在读取完整张图片之后才会做决定

x 不是提前定义的变量；
它就是你在调用模型时传入的数据；
PyTorch 会自动把它传入 forward(self, x) 方法。(def forward(self, x):)

In [4]:
rnn = RNN()
print(rnn)

RNN(
  (rnn): LSTM(28, 64, batch_first=True)
  (out): Linear(in_features=64, out_features=10, bias=True)
)


数据集和数据加载方式决定了 x 的 shape，在深度学习框架（如 PyTorch、TensorFlow）里，图片数据结构一般是 [batch_size, channels, height, width]。

In [ ]:
optimizer = torch.optim.Adam(rnn.parameters(), lr=LR)
loss_fn = nn.CrossEntropyLoss()#target为真实标签，而不是one-hot编码

#x来源于train_loader x的shape是(batch_size, channel（即高度）, height, width)
print(x.shape)

for epoch in range(EPOCH):
    for step, (x, y) in enumerate(train_loader):
        # 将输入数据的形状调整为 (batch, time_step, input_size)
        x = x.view(-1, 28, 28)
        output = rnn(x)
        loss = loss_fn(output, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if step % 100 == 0:
            print(f'Epoch: {epoch}, Step: {step}, Loss: {loss.item()}')

torch.Size([64, 1, 28, 28])
Epoch: 0, Step: 0, Loss: 2.291994571685791
Epoch: 0, Step: 100, Loss: 0.5436904430389404
Epoch: 0, Step: 200, Loss: 0.48724567890167236
Epoch: 0, Step: 300, Loss: 0.13200460374355316
Epoch: 0, Step: 400, Loss: 0.2582266926765442
Epoch: 0, Step: 500, Loss: 0.26670020818710327
Epoch: 0, Step: 600, Loss: 0.23864580690860748
Epoch: 0, Step: 700, Loss: 0.1789989024400711
Epoch: 0, Step: 800, Loss: 0.09383652359247208
Epoch: 0, Step: 900, Loss: 0.03441943973302841


In [7]:
test_output = rnn(test_x[:10].view(-1, 28, 28))
pred_y = torch.max(test_output, 1)[1].data.numpy().squeeze()
print(pred_y,'<-- 预测结果')
print(test_y[:10], '<-- 真实结果')

[7 2 1 0 4 1 4 9 5 9] <-- 预测结果
tensor([7, 2, 1, 0, 4, 1, 4, 9, 5, 9]) <-- 真实结果


test_output 的 shape 是 [batch_size, num_classes]，比如 [10, 10]  
① torch.max(test_output, 1)对 test_output 的每一行（每个样本）找最大值（即哪个类别分数最高）,1 表示“按第1个维度（列/类别方向）”做最大值运算,返回一个元组 (最大值, 最大值的索引)  
②torch.max(test_output, 1)[0] 是最大值（概率/得分）  
torch.max(test_output, 1)[1] 是最大值位置（即预测的类别编号）  
③.squeeze()去掉多余的维度，比如如果shape是 [10, 1]，变成 [10]

## Regression

In [ ]:
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt

torch.manual_seed(1)

TIME_STEP = 10
INPUT_SIZE = 1
LR = 0.02
DOWNLOAD_MNIST = False

In [ ]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        
        self.rnn = nn.RNN(
            input_size = INPUT_SIZE,
            hidden_size = 32,
            num_layers = 1,
            batch_first = True
        )
        self.out = nn.Linear(32, 1)
        
    def forward(self, x, h_state):#上一层的h_state作为输入用来计算着一层的h_state
        #x: (batch, time_step, input_size)
        #h_state: (num_layers, batch, hidden_size)
        #r_out: (batch, time_step, hidden_size)
        r_out, h_state = self.rnn(x, h_state)
        outs = []
        for time_step in range(r_out.size(1)):
            outs.append(self.out(r_out[:, time_step, :]))
        return torch.stack(outs, dim=1), h_state

语音识别时，一个长语音可能会分成好几段输入，处理每一段都需要几个time_step,但是每个time_step都会输出一个h_state，h_state也会随着time_step传递，然后最后一个time_step的h_state，并作为下一个语音片段的第一个time_step的input，所以说h_state就是***记忆***，就像llm的上下文连贯有记忆一样  

图片生成也一样，就是把一张大图分割成若干小图，然后h_state作为线把各个被分割的图片串联起来